In [1]:
from bs4 import BeautifulSoup
import requests
import urllib.request
from urllib.request import Request, urlopen
import json
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import numpy as np

In [2]:
# Function to parse to web
def web_parser(link, headers = {'User-Agent': 'Mozilla/5.0'}):
    req = Request(
    url= link, 
    headers=headers
    )
    webpage = urlopen(req).read()
    soup = BeautifulSoup(webpage, 'html.parser')
    return soup

def get_link(soup):
    links = soup.find('article', class_='comp comp_reviews-pagination querylist-pagination position-').find('ul').find_all('a')
    new_url = 'https://www.airlinequality.com' + links[-1].get('href')
    return new_url

def convert(lst):
   res_dict = {}
   for i in range(0, len(lst), 2):
       res_dict[lst[i]] = lst[i + 1]
   return res_dict

In [3]:
def crawl(soup):
    dataset = []
    count = 1
    while True:
        # Vòng while True này ứng với mỗi page 
        rvs = soup.find_all('article', itemprop='review')
        for i in range(len(rvs)):
            tlist = []
            # Mỗi vòng for ứng với mỗi review trong một page
            rv_id = rvs[i].find('div', class_='body').get('id')
            rv_date = rvs[i].find('meta', itemprop='datePublished').get("content")
            rv_header = rvs[i].find('h2', class_='text_header').get_text()
            rv_text = rvs[i].find('div', class_='text_content', itemprop='reviewBody').get_text()
            rating_element = rvs[i].find('span', itemprop='ratingValue')
            if rating_element is not None:
                # Extract the text of the rating element and convert it to an integer
                overall_rating = int(rating_element.text)
            else:
                overall_rating = np.nan
            # Crawl stats data
            table = rvs[i].find_all('td')
            for j in range(len(table)):
                if table[j].get('class') == ['review-rating-stars', 'stars']:
                    if len(table[j].find_all('span', class_ ="star fill")) != 0:
                        value = table[j].find_all('span', class_ ="star fill")[-1].get_text()
                else:
                    value = table[j].get_text()
                tlist.append(value)
            tdict = convert(tlist)
            tdict['Review id'] = rv_id
            tdict['Review date'] = rv_date
            tdict['Review header'] = rv_header
            tdict['Review'] = rv_text
            tdict['Overall rating'] = overall_rating
            dataset.append(tdict)
        link = get_link(soup)
        new_soup = web_parser(link)
        if new_soup == soup:
            print("Last page")
            break
        else:
            soup = new_soup
            print(f'Finish page {count}')
        count += 1
    return dataset


            
            
            

In [120]:
airline_name = 'frontier-airlines'

In [121]:
airline_url = 'https://www.airlinequality.com/airline-reviews/' + airline_name + '/'
req = Request(
    url= airline_url, 
    headers={'User-Agent': 'Mozilla/5.0'}
)
webpage = urlopen(req).read()
soup = BeautifulSoup(webpage, 'html.parser')

In [122]:
data = crawl(soup)
df = pd.DataFrame(data)
save_path = 'D:/Đề án/Data2/' + airline_name + '.csv'
df.to_csv(save_path, index=False, encoding='utf-8-sig')
print(f'Finish {airline_name}')

Finish page 1
Finish page 2
Finish page 3
Finish page 4
Finish page 5
Finish page 6
Finish page 7
Finish page 8
Finish page 9
Finish page 10
Finish page 11
Finish page 12
Finish page 13
Finish page 14
Finish page 15
Finish page 16
Finish page 17
Finish page 18
Finish page 19
Finish page 20
Finish page 21
Finish page 22
Finish page 23
Finish page 24
Finish page 25
Finish page 26
Finish page 27
Finish page 28
Finish page 29
Finish page 30
Finish page 31
Finish page 32
Finish page 33
Finish page 34
Finish page 35
Finish page 36
Finish page 37
Finish page 38
Finish page 39
Finish page 40
Finish page 41
Finish page 42
Finish page 43
Finish page 44
Finish page 45
Finish page 46
Finish page 47
Finish page 48
Finish page 49
Finish page 50
Finish page 51
Finish page 52
Finish page 53
Finish page 54
Finish page 55
Finish page 56
Finish page 57
Finish page 58
Finish page 59
Finish page 60
Finish page 61
Finish page 62
Finish page 63
Finish page 64
Finish page 65
Finish page 66
Finish page 67
Fini

In [123]:
df

,Type Of Traveller,Seat Type,Route,Date Flown,Seat Comfort,Cabin Staff Service,Ground Service,Value For Money,Recommended,Review id,Review date,Review header,Review,Overall rating,Food & Beverages,Inflight Entertainment,Wifi & Connectivity,Aircraft
0,Couple Leisure,Economy Class,Minneapolis to Orlando,March 2024,1,1,1,1,no,anchor889844,2024-03-02,“It was awful”,Not Verified | Horrible customer service and e...,1.0,NaN,NaN,NaN,NaN
1,Couple Leisure,Economy Class,Denver to Baltimore,February 2024,1,1,1,1,no,anchor889695,2024-02-29,"""customer service was horrible""",Not Verified | The customer service was horr...,1.0,1,1,1,NaN
2,Couple Leisure,Economy Class,Salt Lake City to Tampa Florida,February 2024,1,1,1,1,no,anchor889550,2024-02-27,“such appalling customer service”,✅ Trip Verified | I'm deeply dismayed by the s...,1.0,1,1,1,NaN
3,Solo Leisure,Economy Class,San Juan to Philadelphia,January 2024,1,1,1,1,no,anchor889443,2024-02-26,"""nothing but rudeness""",Not Verified | I was driven to the airport b...,1.0,1,1,1,NaN
4,Couple Leisure,Economy Class,Minneapolis to Orlando,February 2024,NaN,NaN,1,1,no,anchor889213,2024-02-24,"""avoid this airline""",Not Verified | My significant other had knee...,2.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3583,NaN,Economy Class,NaN,NaN,2,3,NaN,3,no,anchor252456,2012-04-02,Frontier Airlines customer review,LIR (Costa Rica)-DEN-LAX. Sat on the runway in...,2.0,Food & Beverages,4,NaN,NaN
3584,NaN,NaN,NaN,NaN,4,4,NaN,4,yes,anchor252457,2012-04-01,Frontier Airlines customer review,Flew round trip from Minneapolis to Las Vegas ...,9.0,4,4,NaN,NaN
3585,NaN,Economy Class,NaN,NaN,5,5,NaN,5,yes,anchor252458,2012-03-29,Frontier Airlines customer review,Flew roundtrip from Denver to San Diego return...,9.0,5,5,NaN,NaN
3586,NaN,Economy Class,NaN,NaN,3,3,NaN,2,no,anchor252459,2012-03-26,Frontier Airlines customer review,ATL-DEN. Arrived on time however the boarding ...,1.0,3,3,NaN,NaN


In [124]:
df['Overall rating'].value_counts()

Overall rating
1.0     2721
2.0      295
10.0     106
3.0       81
8.0       60
9.0       54
5.0       35
4.0       34
7.0       24
6.0       11
Name: count, dtype: int64